    1- test function
    2- MC tests

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
plt.rc('axes', labelsize=10)
plt.rc('legend', fontsize=8)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# 1- test function

In [ ]:
from GPPY.monte_carlo_test_functions import (f_1, f_2, f_3, f_4, f_5, f_6,
                                             exact_integral_f_1, exact_integral_f_2,
                                             exact_integral_f_3, exact_integral_f_4, 
                                             exact_integral_f_5,
                                             exact_integral_f_6,
                                             support_integrands)


In [ ]:
nb_function=5
for d in range(2,8):
    print("for d={}".format(d))
    i_f = [{"f_"+str(i): globals()["exact_integral_f_{}".format(i)](d)} for i in range(1,nb_function+1)]
    print("Exact integral functions=", i_f )

In [ ]:
r=3/4 #radius observation window
x = np.linspace(-r,r, 180)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
z_f_4 = f_4(points)
z_f_5 = f_5(points)
z_f_6 = f_6(points)
fig = plt.figure(figsize=(14, 4))
ax = fig.add_subplot(2, 6, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(2, 6, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(2, 6, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_3)
ax.set_title(r"$f_3$")
ax.view_init(-140, 60)
ax = fig.add_subplot(2, 6, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4)
ax.set_title(r"$f_4$")
ax.view_init(40, 60)
ax = fig.add_subplot(2, 6, 5, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_5, c=z_f_5)
ax.set_title(r"$f_5$")
ax.view_init(-140, 60)
ax = fig.add_subplot(2, 6, 6, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_6, c=z_f_6)
ax.set_title(r"$f_6$")
plt.show()

In [ ]:
r=1 #radius observation window
x = np.atleast_2d(np.linspace(-r,r, 300)).T
z_f_1 = f_1(x)
z_f_2 = f_2(x)
z_f_3 = f_3(x)
z_f_4 = f_4(x)
z_f_5 = f_5(x)
z_f_6 = f_6(x)

fig = plt.figure(figsize=(10, 3))
ax = fig.add_subplot(2, 6, 1)
ax.plot(x, z_f_1 )
ax.set_title(r"$f_1$")
ax = fig.add_subplot(2, 6, 2)
ax.plot(x, z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(2, 6, 3)
ax.plot(x,  z_f_3)
ax.set_title(r"$f_3$")
ax = fig.add_subplot(2, 6, 4)
ax.plot( x, z_f_4)
ax.set_title(r"$f_4$")
ax = fig.add_subplot(2, 6, 5)
ax.plot(x, z_f_5)
ax.set_title(r"$f_5$")
ax = fig.add_subplot(2, 6, 6)
ax.plot(x, z_f_6)
ax.set_title(r"$f_6$")
plt.show()

# 2- MC tests 

In [ ]:
import multiprocessing
from GPPY.monte_carlo_tests_setup import (plot_mc_results, mc_results, 
                                          dataframe_mse_results,
                                          dataframe_residual_test,
                                          dataframe_error_test,
                                          qq_plot_residual
                                         )
multiprocessing.cpu_count()

In [ ]:
nb_sample=100
nb_core = 70
nb_point_list=np.arange(100, 1550, 50).tolist()
print(len(nb_point_list), nb_point_list)
fct_list = [f_1, f_2, f_3, f_4, f_5]
fct_names = ["f_1", "f_2", "f_3", "f_4", "f_5"]
estimators = ["MC", 
              "MCP",
              "MCCV",
              "MCDPP",
              "RQMC"]

### For d=2

In [ ]:
d=2
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d), exact_integral_f_4(d), 
                  exact_integral_f_5(d)]
support_window = support_integrands(d) 

In [ ]:
if __name__ == "__main__":
    mc_results_2d, nb_point_2d = mc_results(d, nb_point_list, nb_sample=nb_sample,
                                             support_window=support_window,
                                            fct_list=fct_list,
                                            fct_names=fct_names,
                                            exact_integrals=exact_integrals,
                                            estimators=estimators, 
                                            core_number=nb_core,
                                            file_name="mc_results_2d.pickle"
                                            )

In [ ]:
dataframe_mse_results(mc_results=mc_results_2d,
                      fct_names=fct_names, 
                      exact_integrals=exact_integrals, 
                      nb_sample=nb_sample)

In [ ]:

with open('mc_results_2d.pickle', 'wb') as handle:
    pickle.dump(mc_results_2d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# load saved data
with open('mc_results_2d.pickle', 'rb') as file:
    mc_results_2d = pickle.load(file)

#### Variance study

In [ ]:
plot_mc_results(d, mc_results_2d, nb_point_list, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                #save_fig="mc_2_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_2d, 
                        nb_point_list=nb_point_list, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
qq_plot_residual(mc_list=mc_results_2d, 
                        nb_point_list=nb_point_list, 
                        fct_names=fct_names,
                save_fig="qq_plot_2d.pdf")

In [ ]:
# Q-Q plot
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Create a Q-Q plot
fig, ax = plt.subplots(figsize=(5, 5))
sm.qqplot(x, line='45', ax=ax)

# Customize the plot
ax.set_title('Normal Q-Q Plot')
ax.set_xlabel('Theoretical Quantiles')
ax.set_ylabel('Sample Quantiles')
ax.grid(True)

# Show the plot
plt.show()


#### Error study

In [ ]:
plot_mc_results(d, mc_results_2d, nb_point_list, nb_sample, 
                fct_list=fct_list, 
                fct_names=fct_names,
                error_type="Error",
                plot_error=True,
                plot_std=False,
                log_scale=False,
                #save_fig="mc_2_d.pdf"
               )

In [ ]:
for name in fct_names:
    print("For "+ name + ":")
    df_error = dataframe_error_test(mc_results_2d, nb_point_list, 
                     fct_name=name,
                     type_mc_test="MCP")
    display(df_error)

### For d=3

In [ ]:
estimators = ["MC", 
              "MCP",
              "MCCV",
              "MCDPP",
              "RQMC"]

In [ ]:
d=3
exact_integrals= [exact_integral_f_1(d), exact_integral_f_2(d),
                  exact_integral_f_3(d), exact_integral_f_4(d), 
                  exact_integral_f_5(d)]
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_3d, nb_point_3d = mc_results(d, nb_point_list, 
                                            nb_sample=nb_sample,
                                             support_window=support_window,
                                            fct_list=fct_list,
                                            fct_names=fct_names,
                                            exact_integrals=exact_integrals,
                                            estimators=estimators, 
                                            core_number=nb_core)

In [ ]:
for name in fct_names:
    print("For "+ name + ":")
    df_error = dataframe_error_test(mc_results_3d, nb_point_list, 
                     fct_name=name,
                     type_mc_test="MCP")
    display(df_error)

#### variance study

In [ ]:
plot_mc_results(d, mc_results_3d, nb_point_list, nb_sample, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                #save_fig="mc_3_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_3d, 
                        nb_point_list=nb_point_list, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
qq_plot_residual(mc_list=mc_results_3d, 
                        nb_point_list=nb_point_list, 
                        fct_names=fct_names,
                save_fig="qq_plot_3d.pdf")

#### Error study

In [ ]:
plot_mc_results(d, mc_results_3d, nb_point_list, nb_sample, 
                fct_list=fct_list, 
                fct_names=fct_names,
                error_type="Error",
                plot_error=True,
                plot_std=False,
                log_scale=False,
                #save_fig="mc_2_d.pdf"
               )

In [ ]:
dataframe_error_test(mc_results_4d, nb_point_list, 
                     fct_name=fct_names,
                     type_mc_test="MCP")

### For d=4

In [ ]:
d=4
exact_integrals= [exact_integral_f_1(d), exact_integral_f_2(d),
                  exact_integral_f_3(d), exact_integral_f_4(d), 
                  exact_integral_f_5(d)]
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_4d, nb_point_4d = mc_results(d, nb_point_list, nb_sample=nb_sample,
                                             support_window=support_window,
                                            fct_list=fct_list,
                                            fct_names=fct_names,
                                            exact_integrals=exact_integrals,
                                            estimators=estimators, 
                                            core_number=nb_core)

In [ ]:
dataframe_mse_results(mc_results=mc_results_4d,
                      fct_names=fct_names, 
                      exact_integrals=exact_integrals, 
                      nb_sample=nb_sample)

#### Variance study

In [ ]:
plot_mc_results(d, mc_results_4d, nb_point_list, nb_sample, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                #save_fig="mc_4_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_4d, 
                        nb_point_list=nb_point_list, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
qq_plot_residual(mc_list=mc_results_4d, 
                        nb_point_list=nb_point_list, 
                        fct_names=fct_names,
                save_fig="qq_plot_4d.pdf")

#### Error study

In [ ]:

plot_mc_results(d, mc_results_4d, nb_point_list, nb_sample, 
                fct_list=fct_list, 
                fct_names=fct_names,
                error_type="Error",
                plot_error=True,
                plot_std=False,
                log_scale=False,
                #save_fig="mc_4_d.pdf"
               )

In [ ]:
for name in fct_names:
    print("For "+ name + ":")
    df_error = dataframe_error_test(mc_results_4d, nb_point_list, 
                     fct_name=name,
                     type_mc_test="MCP")
    display(df_error)

In [ ]:
# accelrating computational time by reducing the points used to push in B(0,R) from B(0, 2R)
# into B(0, R + add_r_push)
if __name__ == "__main__":
    mc_results_4d, nb_point_4d = mc_results(d, nb_point_list, nb_sample=nb_sample,
                                             support_window=support_window,
                                            fct_list=fct_list,
                                            fct_names=fct_names,
                                            exact_integrals=exact_integrals,
                                            estimators=estimators, 
                                            core_number=nb_core, 
                                           add_r_push=0.2)


In [ ]:
dataframe_mse_results(mc_results=mc_results_4d,
                      fct_names=fct_names, 
                      exact_integrals=exact_integrals, 
                      nb_sample=nb_sample)